In [31]:
import os
import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [51]:
train_dir = '../Nails research paper/nails images/new texture imgs/new texture imgs/brittle nails'
# test_dir = '../input/chest-xray-covid19-pneumonia/Data/test/*'
# train2_covid_dir = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'


In [52]:
SIZE = 140
BATCH_SIZE = 64
TARGET_SIZE = (SIZE,SIZE)
EPOCH_NUM = 140

In [64]:
categories_dict = {
  0: "brittle nails",
  1: "nail pitting",
  2: "rough nails",
  3: "vertical nails"
}

In [65]:
train_images = []
train_labels = [] 
label = 0
#Importing the first training dataset

for directory_path in glob.glob(train_dir):
#     assert categories_dict[label] == os.path.normpath(directory_path).split(os.path.sep)[-1]
#     print(categories_dict[label])
    counter = 1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        if(counter%200==0): print(counter,"images loaded")
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        train_images.append(img)
        train_labels.append(label)
        counter+=1
        if(counter%1500==0): break
    
    print(counter,"images loaded")
    label +=1
    
#Importing the additional training dataset

print("additional",categories_dict[2],"data")    
# addit_counter = 1
# for img_path in glob.glob(os.path.join(train2_covid_dir, "*.png")):
#     if(addit_counter%200==0): print(addit_counter,"images loaded")
#     img = cv2.imread(img_path, 0)
#     img = cv2.resize(img, TARGET_SIZE)
#     train_images.append(img)
#     train_labels.append(2)
#     addit_counter+=1
#     if(addit_counter%1000==0): break
# print(addit_counter,"images loaded")

x_train = np.array(train_images)
# y_train = to_categorical(train_labels, 3)

13 images loaded
additional rough nails data


In [66]:
test_images = []
test_labels = []
label = 0
#importing the testing dataset

for directory_path in glob.glob(train_dir):
#     assert categories_dict[label] == os.path.normpath(directory_path).split(os.path.sep)[-1]
    print(categories_dict[label])
    counter = 1
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        if(counter%100==0): print(counter, "images loaded")
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        test_images.append(img)
        test_labels.append(label)
        counter+=1
    
    print(counter,"images loaded")
    label +=1

test_images = np.array(test_images)
# test_labels = to_categorical(test_labels, 3)

brittle nails
13 images loaded


In [67]:
train_test_split(train_images)
train_images, val_labels = train_test_split(x_train, test_size=0.15, random_state=69)

In [68]:
print("train:",train_images.shape[0],", test:",test_images.shape[0],)

train: 10 , test: 12


In [69]:
def feature_extractor(images):
    image_dataset = pd.DataFrame()
    for image in images:   
        df = pd.DataFrame()
        
        #greycomatrix(image, distances, angles, levels=256, symmetric=False, normed=False)
        #distances - List of pixel pair distance offsets.
        #angles - List of pixel pair angles in radians.
        
        #5 configuration for the grey-level co-occurrence matrix calculation
        dists = [[1],[3],[5],[3],[3]]
        angles = [[0],[0],[0],[np.pi/4],[np.pi/2]]
        
        for n ,(dist, angle) in enumerate(zip(dists, angles)):
        
            GLCM = greycomatrix(image, dist, angle)       
            GLCM_Energy = greycoprops(GLCM, 'energy')[0]
            df['Energy'+str(n)] = GLCM_Energy
            GLCM_corr = greycoprops(GLCM, 'correlation')[0]
            df['Corr'+str(n)] = GLCM_corr       
            GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
            df['Diss_sim'+str(n)] = GLCM_diss       
            GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
            df['Homogen'+str(n)] = GLCM_hom       
            GLCM_contr = greycoprops(GLCM, 'contrast')[0]
            df['Contrast'+str(n)] = GLCM_contr

        image_dataset = image_dataset.append(df)
        
    return image_dataset

In [70]:
train_extr_features = feature_extractor(train_images)

In [71]:
test_extr_features = feature_extractor(test_images)

In [72]:
val_extr_features = feature_extractor(val_images)

NameError: name 'val_images' is not defined

In [73]:
type(train_images)
# convert from integers to floats
train_images_norm = train_images.astype('float32')
# test_images_norm = test_images.astype('float32')
# val_images_norm = val_images.astype('float32')
# normalize to the range 0-1
train_images_norm /= 255.0
# test_images_norm /= 255.0
# val_images_norm /= 255.0

In [74]:
pd.set_option("display.max_columns", None)
train_extr_features

,Energy0,Corr0,Diss_sim0,Homogen0,Contrast0,Energy1,Corr1,Diss_sim1,Homogen1,Contrast1,Energy2,Corr2,Diss_sim2,Homogen2,Contrast2,Energy3,Corr3,Diss_sim3,Homogen3,Contrast3,Energy4,Corr4,Diss_sim4,Homogen4,Contrast4
0,0.036387,0.997054,1.825026,0.434078,6.554728,0.024126,0.979890,4.871220,0.208712,45.187278,0.021248,0.959694,7.054286,0.145795,92.190159,0.023815,0.977473,4.800462,0.232814,49.459462,0.023032,0.970742,5.208603,0.225918,62.977633
0,0.039666,0.992893,2.251901,0.374165,9.783145,0.026720,0.954026,5.762982,0.177799,62.913034,0.024066,0.912262,7.983968,0.136230,120.222169,0.028546,0.961698,5.065532,0.210721,52.541903,0.032227,0.970512,3.831126,0.295691,39.778676
0,0.032800,0.971017,4.149126,0.308453,47.793834,0.024897,0.868933,8.978624,0.173374,216.660271,0.021974,0.764352,12.581217,0.129672,390.705661,0.024281,0.850816,9.599086,0.166497,246.608433,0.025699,0.881925,8.303180,0.183364,194.725391
0,0.048436,0.987131,2.662539,0.414163,19.045272,0.032422,0.915986,6.856048,0.225622,121.044995,0.028009,0.838604,9.683862,0.170791,228.779735,0.032218,0.934218,6.054505,0.235283,95.176434,0.033665,0.960321,4.759906,0.286296,59.488060
0,0.049538,0.991938,1.892652,0.455614,7.933864,0.032719,0.949469,4.882377,0.236535,49.024296,0.028414,0.905173,6.758148,0.174510,91.452963,0.032620,0.958480,4.280193,0.266973,39.665511,0.034562,0.971843,3.646455,0.281782,27.797550
0,0.059117,0.992727,1.437616,0.532199,4.851593,0.037940,0.945821,4.038843,0.286941,36.373253,0.031208,0.877308,6.188466,0.200882,83.032698,0.040078,0.964884,3.305976,0.313840,23.613684,0.041822,0.973517,2.848488,0.321588,17.599791
0,0.033552,0.994647,2.417215,0.379148,11.870452,0.023166,0.968948,5.842649,0.195080,68.461522,0.021233,0.943386,7.770423,0.155018,124.572116,0.023308,0.957678,6.056291,0.197678,91.689246,0.023167,0.948331,6.454536,0.196311,115.717518
0,0.026919,0.979362,5.316855,0.232859,72.362795,0.020905,0.926101,10.223045,0.126153,258.829510,0.018979,0.882687,13.578624,0.081823,411.008783,0.021364,0.944258,9.049464,0.147622,196.505146,0.023684,0.936612,8.689051,0.196413,225.511157
0,0.038047,0.993114,2.867420,0.364519,21.444707,0.025189,0.951305,7.507508,0.181066,147.022941,0.022146,0.897714,10.794233,0.131661,304.722275,0.025711,0.956888,6.889414,0.184474,116.793741,0.026615,0.966348,5.763399,0.211041,92.663816
0,0.029128,0.990919,4.514440,0.290450,83.417009,0.022034,0.949451,11.346663,0.145604,453.993274,0.019626,0.907168,16.204127,0.111354,822.489206,0.021902,0.941962,11.114734,0.142733,525.967496,0.023832,0.948054,9.045151,0.194875,485.379145
